In [1]:
# - remove last categories
# - change opacity, stroke
# - axis: bottom
# - binsize
# - title: add data type as title/remove title
# - legend: add/remove
# - remove axis

In [2]:
import os
import json
import random

In [ ]:
data_path = "../data"
indexed_set = [file.split(".png")[0] for file in os.listdir(os.path.join(data_path, "indexed/single_chart"))]
specs_dir = os.path.join(data_path, "unified/specs")

In [4]:
options = ["categories", "title", "binSize", "axis", "legend"]

In [5]:
## Retrieve possible changes

def exists_in_spec(data, attribute):
    if attribute in data:
        return True
    return False

def check_more_than_1_category(obj):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if key == "categories":
                if len(obj[key]) > 1:
                    return True
                else:
                    return False
            if check_more_than_1_category(value):
                return True
    elif isinstance(obj, list):
        for item in obj:
            if check_more_than_1_category(item):
                return True
            
def choose_option(data, data_json):
    options_existing = []
    for option in options: 
        if exists_in_spec(data, option):
            if option == "categories":
                # categories also needs to be more than 1
                if check_more_than_1_category(data_json):
                    options_existing.append(option)
            elif option == "axis":
                options_existing.extend(["changeAxis", "removeAxis"])
            elif option == "legend":
                options_existing.extend(["changeLegend", "removeLegend"])
            else:
                options_existing.append(option)
        
    if len(options_existing) > 0: 
        option_chosen = random.choice(options_existing)
    else: 
        # print("NO POSSIBLE CHANGES")
        option_chosen = None
    return option_chosen

In [6]:
## Execute possible change

def change_option(obj, option, modify_fn, remove=False):
    if isinstance(obj, dict):
        for key in list(obj.keys()):
            if key == option:
                if remove: 
                    del obj[key]
                else:
                    obj[key] = modify_fn(obj[key])
            else:
                change_option(obj[key], option, modify_fn, remove)
    elif isinstance(obj, list):
        for item in obj:
            change_option(item, option, modify_fn, remove)

def change_categories(obj):
    change_option(obj, "categories", lambda x: x[:-1] if len(x) > 1 else print("WARNING!"))

def remove_title(obj):
    change_option(obj, "title", lambda x: None, remove=True)
    change_option(obj, "subtitle", lambda x: None, remove=True)

def change_binSize(obj):
    change_option(obj, "binSize", lambda x: x * 2)

def change_axis(obj):
    change_option(obj, "axis", lambda x: "top" if x == "bottom" else "bottom")

def remove_axis(obj):
    change_option(obj, "axis", lambda x: None, remove=True)

def change_legend(obj):
    change_option(obj, "legend", lambda x: not x)

def remove_legend(obj):
    change_option(obj, "legend", lambda x: None, remove=True)

def change_spec(data_json, option_chosen):
    match option_chosen:
        case "categories":
            change_categories(data_json)
        case "title":
            remove_title(data_json)
        case "binSize":
            change_binSize(data_json)
        case "changeAxis":
            change_axis(data_json)
        case "removeAxis":
            remove_axis(data_json)
        case "changeLegend":
            change_legend(data_json)
        case "removeLegend":
            remove_legend(data_json)

In [7]:
overview = {}

for file in indexed_set: 
    file_name = f"{file}.json"
    
    with open(os.path.join(specs_dir, file_name), "r") as f:
        data = f.read()
        data_json = json.loads(data)

    data_json_copy = json.loads(json.dumps(data_json))
    option_chosen = choose_option(data, data_json)

    if option_chosen == None: 
        print("skipping", file_name)
        continue

    overview[file_name] = option_chosen
    change_spec(data_json, option_chosen)

    if data_json == data_json_copy:
        print("error", file_name, "with", option_chosen)
        continue
    
    with open(os.path.join(data_path, "validation/specs", file_name), "w") as f:
        f.write(json.dumps(data_json))

skipping band_connection_p_0_sw_1_0_s_1_0_oc.json
skipping band_connection_p_0_sw_0_7_s_0_7_cc_2.json
skipping responsive-ideogram_p_0_sw_1_0_s_1_0.json
skipping breast_cancer_sw_2_0_s_1_2_oc.json
skipping band_connection_p_0_sw_1_2_s_1_0_cc_1.json
skipping breast_cancer_circular_s_2_0_oc.json


In [8]:
overview

{'stratified-area_p_0_sw_0_7_s_0_7_cc_0.json': 'removeAxis',
 'gene_annotation_single_1_s_1_0_oc.json': 'title',
 'gene_annotation_single_6_s_1_0_oc.json': 'title',
 'circular-heat_p_0_sw_1_0_s_1_0_cc_0.json': 'title',
 'multi_layer_circular_m_3_sw_0_7_s_0_7_cc_0.json': 'changeLegend',
 'responsive-circular_p_0_sw_1_2_s_1_2_cc_0.json': 'removeAxis',
 'stacked-bar_p_0_sw_1_2_s_1_2_oc.json': 'changeAxis',
 'hic_matrix_sw_1_0_s_1_0_oc_sw_0_7_s_0_7_cc_0.json': 'changeAxis',
 'rule-mark_p_0_sw_0_7_s_0_7_cc_0.json': 'binSize',
 'multi_layer_circular_m_3_s_2_0_oc.json': 'removeAxis',
 'EX_SPEC_RESPONSIVE_COMPARATIVE_MATRICES_sw_0_7_s_0_7_p_0_sw_0_7_s_1_0_oc.json': 'removeAxis',
 'LINE_sw_0_7_s_1_2_cc_2.json': 'title',
 'stratified-line_p_0_sw_1_2_s_1_2_oc.json': 'changeAxis',
 'SEMANTIC_ZOOM_SEQUENCE_sw_1_2_s_1_2_oc.json': 'removeAxis',
 'EX_SPEC_RESPONSIVE_COMPARATIVE_MATRICES_sw_0_7_s_0_7_p_0_sw_1_0_s_1_0_cc_0.json': 'changeLegend',
 'multi_layer_circular_m_0_s_0_5_cc_0.json': 'changeAxis',